In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import d2lzh_pytorch as d2l

# 获取和读取数据集

In [3]:
train_data = pd.read_csv('house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('house-prices-advanced-regression-techniques/test.csv')
print(train_data.shape) # 80个特征 1个标签
print(test_data.shape)

(1460, 81)
(1459, 80)


In [4]:
train_data.iloc[0:4, [0, 1, 2, 3, -3, -2, -1]]

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,208500
1,2,20,RL,80.0,WD,Normal,181500
2,3,60,RL,68.0,WD,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,140000


In [7]:
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:])) # 删除掉第一个id数据，对我们的训练是没有意义的,并将训练集和测试集上下连接起来
print(all_features)

      MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0             60       RL         65.0     8450   Pave   NaN      Reg   
1             20       RL         80.0     9600   Pave   NaN      Reg   
2             60       RL         68.0    11250   Pave   NaN      IR1   
3             70       RL         60.0     9550   Pave   NaN      IR1   
4             60       RL         84.0    14260   Pave   NaN      IR1   
...          ...      ...          ...      ...    ...   ...      ...   
1454         160       RM         21.0     1936   Pave   NaN      Reg   
1455         160       RM         21.0     1894   Pave   NaN      Reg   
1456          20       RL        160.0    20000   Pave   NaN      Reg   
1457          85       RL         62.0    10441   Pave   NaN      Reg   
1458          60       RL         74.0     9627   Pave   NaN      Reg   

     LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC  Fence  \
0            Lvl    AllPub    Inside  ...  

# 预处理数据

In [8]:
# 对数据特征进行标准化，将缺失值切换成特征的均值
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(lambda x: (x - x.mean()) / (x.std()))

all_features = all_features.fillna(0)   # 标准化后，每个特征的均值变成0，所以用0来代替缺失值

In [9]:
# 将离散坐标转化为指示特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape  # 这里是353， 但是我看教程给的是353

(2919, 353)

[[0.06731987999872384 -0.1844426168571562 -0.21784136849269334 ... True
  False False]
 [-0.8734663783676423 0.4580958808711674 -0.07203174114674873 ... True
  False False]
 [0.06731987999872384 -0.05593491731149148 0.13717337634960655 ... True
  False False]
 ...
 [0.30251644459031535 -0.1416067170086013 -0.14278110815460707 ... True
  False False]
 [-0.8734663783676423 -0.05593491731149148 -0.05719719645155263 ... True
  False False]
 [-0.8734663783676423 0.24391638162839288 -0.029303180785371925 ... True
  False False]]


In [12]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float)
train_labels = torch.tensor(train_data.SalePrice.values, dtype=torch.float).view(-1, 1)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

# 做的非常痛苦，我觉得还是得进一步了解Python了再来回头做